In [1]:
import os
deepvariant_output_path = "/Users/joeyrich/Desktop/local/RLSRWP_2025/scripts/trash/deepvariant_output"
os.makedirs(deepvariant_output_path, exist_ok=True)
os.chdir(deepvariant_output_path)

reference_genome = "/Users/joeyrich/Documents/Caltech/Pachter/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa"

In [12]:
import shutil

if shutil.which("docker"):
    print("Docker is installed.")
else:
    print("Docker is NOT installed.")


Docker is installed.


In [2]:
if not os.path.exists(reference_genome):
    !gget ref -w dna -r 93 --out_dir {os.path.dirname(reference_genome)} -d human_grch37 && gunzip {reference_genome}
reference_genome_index = f"{reference_genome}.fai"
if not os.path.exists(reference_genome_index):
    !samtools faidx {reference_genome}

In [3]:
!mkdir -p data benchmark reference model output happy

In [10]:
#!!! DELETE
FTPDIR="ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/ChineseTrio/HG005_NA24631_son/NISTv4.2.1/GRCh38"

!curl -L {FTPDIR}/HG005_GRCh38_1_22_v4.2.1_benchmark.bed > benchmark/HG005_GRCh38_1_22_v4.2.1_benchmark.bed
!curl -L {FTPDIR}/HG005_GRCh38_1_22_v4.2.1_benchmark.vcf.gz > benchmark/HG005_GRCh38_1_22_v4.2.1_benchmark.vcf.gz
!curl -L {FTPDIR}/HG005_GRCh38_1_22_v4.2.1_benchmark.vcf.gz.tbi > benchmark/HG005_GRCh38_1_22_v4.2.1_benchmark.vcf.gz.tbi

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   346  100   346    0     0   1610      0 --:--:-- --:--:-- --:--:--  1632
100 17.7M  100 17.7M    0     0  2560k      0  0:00:07  0:00:07 --:--:-- 3142k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   349  100   349    0     0   2536      0 --:--:-- --:--:-- --:--:--  2566
100  132M  100  132M    0     0  11.5M      0  0:00:11  0:00:11 --:--:-- 13.9M8 11.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   353  100   353    0     0   2607      0 --:--:-- --:--:-- --:--:--  2654
100 1577k  100 1577k    0     0  1791k      0 --:--:-- --:--:-- --:--:-- 2900k


In [ ]:
#!!! DELETE
curl -L https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_41/gencode.v41.basic.annotation.gff3.gz > data/gencode.v41.basic.annotation.gff3.gz

# Extract chr20 CDS regions and convert to bed file.
gzip -dc data/gencode.v41.basic.annotation.gff3.gz | \
awk -v OFS='\t' '$1 == "chr20" && $3 == "CDS"  && $4 < $5 { print $1, $4, $5, "CDS" }' | \
awk '!dup[$0]++' > data/chr20_CDS.bed

In [11]:
#!!! DELETE
HTTPDIR="https://storage.googleapis.com/brain-genomics-public/research/sequencing/grch38/bam/rna/illumina/mrna"

!curl -L {HTTPDIR}/hg005_gm26107.mrna.grch38.bam > data/hg005_gm26107.mrna.grch38.bam
!curl -L {HTTPDIR}/hg005_gm26107.mrna.grch38.bam.bai > data/hg005_gm26107.mrna.grch38.bam.bai

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9042M  100 9042M    0     0  13.4M      0  0:11:14  0:11:14 --:--:-- 15.0M0     0  14.3M      0  0:10:30  0:00:54  0:09:36 14.9M  0:01:04  0:09:21 15.3M    0  0:10:40  0:01:26  0:09:14 14.1M    0  0:10:40  0:01:36  0:09:04 14.8M10:49  0:01:45  0:09:04 10.4M 0  13.7M      0  0:10:57  0:02:21  0:08:36 14.2M     0  13.7M      0  0:10:56  0:02:23  0:08:33 14.6M0     0  13.7M      0  0:10:55  0:02:25  0:08:30 14.9M0:10:54  0:02:28  0:08:26 14.8M.7M      0  0:10:56  0:02:30  0:08:26 13.3M4 12.6M.0M13.6M      0  0:11:03  0:02:43  0:08:20 10.2M7  0:08:31 10.8M  0  13.1M      0  0:11:27  0:03:08  0:08:19 11.7M0     0  13.0M      0  0:11:33  0:03:21  0:08:12 8065k  0:04:06  0:07:25 14.6M:22  0:04:46  0:06:36 14.2M 0     0  13.3M      0  0:11:16  0:05:14  0:06:02 15.0MM      0  0:11:15  0:05:16  0:05:59 15.0M     0  13.4M      0  0:11:11

In [ ]:
#!!! DELETE
HTTPDIR=https://storage.googleapis.com/brain-genomics-public/research/sequencing/grch38/bam/rna/illumina/mrna

curl -L ${HTTPDIR}/hg005_gm26107.mrna.grch38.bam > data/hg005_gm26107.mrna.grch38.bam
curl -L ${HTTPDIR}/hg005_gm26107.mrna.grch38.bam.bai > data/hg005_gm26107.mrna.grch38.bam.bai

In [ ]:
#!!! DELETE
# Generate a coverage file, and filter for 3x.
!sudo docker run \
  -v "$(pwd):$(pwd)" \
  -w $(pwd) \
  -it quay.io/biocontainers/mosdepth:0.3.1--h4dc83fb_1 \
  mosdepth \
    --threads $(nproc) \
    data/hg005_coverage \
    data/hg005_gm26107.mrna.grch38.bam

In [ ]:
#!!! DELETE
!sudo docker run \
  -v "$(pwd):$(pwd)" \
  -w $(pwd) \
  -it quay.io/biocontainers/bedtools:2.23.0--h5b5514e_6 \
  /bin/bash

In [ ]:
#!!! DELETE
# (Run within the bedtools container)
bedtools intersect \
-a data/hg005_3x.bed \
-b data/chr20_CDS.bed > data/chr20_CDS_3x.bed

# We will also intersect this file with confident GIAB regions
bedtools intersect \
-a benchmark/HG005_GRCh38_1_22_v4.2.1_benchmark.bed \
-b data/chr20_CDS_3x.bed > benchmark/chr20_CDS_3x.benchmark_regions.bed

In [ ]:
curl https://storage.googleapis.com/deepvariant/models/DeepVariant/1.4.0/DeepVariant-inception_v3-1.4.0+data-rnaseq_standard/model.ckpt.data-00000-of-00001 > model/model.ckpt.data-00000-of-00001
curl https://storage.googleapis.com/deepvariant/models/DeepVariant/1.4.0/DeepVariant-inception_v3-1.4.0+data-rnaseq_standard/model.ckpt.example_info.json > model/model.ckpt.example_info.json
curl https://storage.googleapis.com/deepvariant/models/DeepVariant/1.4.0/DeepVariant-inception_v3-1.4.0+data-rnaseq_standard/model.ckpt.index > model/model.ckpt.index
curl https://storage.googleapis.com/deepvariant/models/DeepVariant/1.4.0/DeepVariant-inception_v3-1.4.0+data-rnaseq_standard/model.ckpt.meta > model/model.ckpt.meta

In [ ]:
BIN_VERSION="1.4.0"

sudo docker run \
  -v "$(pwd):$(pwd)" \
  -w $(pwd) \
  google/deepvariant:"${BIN_VERSION}" \
  run_deepvariant \
    --model_type=WES \
    --customized_model=model/model.ckpt \
    --ref={reference_genome} \
    --reads=data/hg005_gm26107.mrna.grch38.bam \
    --output_vcf=output/HG005.output.vcf.gz \
    --num_shards=$(nproc) \
    --regions=data/chr20_CDS_3x.bed \
    --make_examples_extra_args="split_skip_reads=true,channel_list='BASE_CHANNELS'" \
    --intermediate_results_dir output/intermediate_results_dir